In [1]:
# @title 1. Install Apache Airflow & Setup Environment
import os

# 1. Install Airflow and pyngrok (for the UI tunnel)
print("⏳ Installing Apache Airflow (this takes ~1 min)...")
!pip install -q apache-airflow==2.7.2 pyngrok pandas

# 2. Set Airflow Home directory
os.environ["AIRFLOW_HOME"] = "/content/airflow"

# 3. Initialize the Database (SQLite)
print("⚙️ Initializing Database...")
!airflow db init > /dev/null 2>&1

# 4. Create an Admin User for the UI
print("👤 Creating Admin User...")
!airflow users create \
    --username admin \
    --firstname System \
    --lastname Admin \
    --role Admin \
    --email admin@example.com \
    --password admin \
    > /dev/null 2>&1

# 5. Create directory for DAGs (Workflows)
!mkdir -p /content/airflow/dags

print("✅ Airflow Installed & Configured.")
print("👉 Username: admin")
print("👉 Password: admin")

⏳ Installing Apache Airflow (this takes ~1 min)...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.3/591.3 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 351.2/351.2 kB 15.4 MB/s eta 0:00

In [2]:
# @title 2. Define the Orchestration Logic (The DAG)
# We write this python code into the Airflow DAGs folder

dag_content = """
from airflow import DAG
from airflow.operators.python import PythonOperator
from airflow.operators.bash import BashOperator
from datetime import datetime, timedelta
import random
import time
import json

# --- SIMULATED BUSINESS LOGIC ---

def generate_order(**context):
    order_id = f"ORD-{random.randint(10000, 99999)}"
    amount = random.randint(50, 500)
    print(f"🛒 Order Generated: {order_id} for ${amount}")

    # Pass data to next task via XCom (Airflow's internal data sharing)
    context['ti'].xcom_push(key='order_data', value={'id': order_id, 'amount': amount})

def process_payment(**context):
    # Get data from previous task
    order_data = context['ti'].xcom_pull(key='order_data', task_ids='generate_order_task')
    order_id = order_data['id']
    print(f"💳 Processing payment for {order_id}...")

    time.sleep(1) # Simulate API call

    # Simulate random failure to show how Orchestration handles errors
    if random.random() < 0.2:
        raise ValueError("❌ Payment Gateway Timeout! (Simulated Failure)")

    print("✅ Payment Authorized.")
    return "PAID"

def check_inventory(**context):
    order_data = context['ti'].xcom_pull(key='order_data', task_ids='generate_order_task')
    print(f"📦 Checking warehouse for {order_data['id']}...")
    time.sleep(1)
    print("✅ Stock Allocated.")

def ship_order(**context):
    order_data = context['ti'].xcom_pull(key='order_data', task_ids='generate_order_task')
    tracking_num = f"TRK-{random.randint(100,999)}"
    print(f"🚚 Order {order_data['id']} Shipped! Tracking: {tracking_num}")

# --- ORCHESTRATION DEFINITION (DAG) ---

default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'start_date': datetime(2023, 1, 1),
    'retries': 2,                     # REAL ORCHESTRATION: Auto-retry on failure
    'retry_delay': timedelta(seconds=5),
}

with DAG(
    'ecommerce_fulfillment_pipeline',
    default_args=default_args,
    description='A simulated E-Commerce Saga',
    schedule_interval=timedelta(seconds=30), # Run every 30 seconds automatically
    catchup=False,
    tags=['production', 'sales'],
) as dag:

    # 1. Define Tasks
    t1 = PythonOperator(
        task_id='generate_order_task',
        python_callable=generate_order,
    )

    t2 = PythonOperator(
        task_id='process_payment_task',
        python_callable=process_payment,
    )

    t3 = PythonOperator(
        task_id='check_inventory_task',
        python_callable=check_inventory,
    )

    t4 = PythonOperator(
        task_id='ship_order_task',
        python_callable=ship_order,
    )

    t5 = BashOperator(
        task_id='notify_customer',
        bash_command='echo "📧 Email sent to customer!"',
    )

    # 2. Define The Flow (The Orchestration)
    # Order -> Payment -> Inventory -> Shipping -> Notification
    t1 >> t2 >> t3 >> t4 >> t5

"""

# Write the file to the Airflow DAGs directory
with open("/content/airflow/dags/ecommerce_pipeline.py", "w") as f:
    f.write(dag_content)

print("✅ DAG file created at /content/airflow/dags/ecommerce_pipeline.py")

✅ DAG file created at /content/airflow/dags/ecommerce_pipeline.py


In [3]:
# @title 3. Start Airflow Webserver & Scheduler
import subprocess
import time
from pyngrok import ngrok, conf

# --- USER INPUT REQUIRED ---
NGROK_AUTH_TOKEN = "2lByyY668AQEl2B574HOizPVWze_   3roGxEj1UQdg2rXyx7tH8" # @param {type:"string"}

if NGROK_AUTH_TOKEN == "PASTE_YOUR_NGROK_TOKEN_HERE" or NGROK_AUTH_TOKEN == "":
    print("❌ Error: You must paste your Ngrok Authtoken above to see the UI.")
    print("get it for free at https://dashboard.ngrok.com/get-started/your-authtoken")
else:
    # 1. Authenticate ngrok
    conf.get_default().auth_token = NGROK_AUTH_TOKEN

    # 2. Kill any old processes
    !pkill -f airflow

    # 3. Start Scheduler (The Heart)
    print("💓 Starting Airflow Scheduler (Background)...")
    subprocess.Popen(["airflow", "scheduler"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # 4. Start Webserver (The UI)
    print("🖥️ Starting Airflow Webserver (Background)...")
    subprocess.Popen(["airflow", "webserver", "--port", "8080"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # 5. Create Tunnel
    print("🔗 Creating secure tunnel...")
    time.sleep(10) # Wait for webserver to boot
    try:
        public_url = ngrok.connect(8080).public_url
        print("\n" + "="*50)
        print(f"🚀 AIRFLOW UI IS LIVE: {public_url}")
        print("="*50)
        print("1. Click the link above.")
        print("2. Login with -> User: admin | Pass: admin")
        print("3. Toggle the 'ecommerce_fulfillment_pipeline' switch to ON (Left side).")
        print("4. Click the DAG name to see the Graph/Grid view.")
        print("="*50)
    except Exception as e:
        print(f"⚠️ Error opening tunnel: {e}")

💓 Starting Airflow Scheduler (Background)...
🖥️ Starting Airflow Webserver (Background)...
🔗 Creating secure tunnel...

🚀 AIRFLOW UI IS LIVE: https://11892bd1ee23.ngrok-free.app
1. Click the link above.
2. Login with -> User: admin | Pass: admin
3. Toggle the 'ecommerce_fulfillment_pipeline' switch to ON (Left side).
4. Click the DAG name to see the Graph/Grid view.


In [4]:
# @title 1. Fix Dependencies & Install Airflow
import sys
import os

# 1. Uninstall conflicting libraries provided by Colab defaults
print("🧹 Cleaning up conflicting libraries...")
!pip uninstall -y sqlalchemy opentelemetry-api opentelemetry-sdk opentelemetry-exporter-gcp-logging google-adk ipython-sql > /dev/null 2>&1

# 2. Install Airflow using the OFFICIAL constraints file
# This forces the versions to match exactly what Airflow needs
print("⏳ Installing Apache Airflow (Robust Mode)...")
AIRFLOW_VERSION = '2.7.2'
PYTHON_VERSION = f"{sys.version_info.major}.{sys.version_info.minor}"
CONSTRAINT_URL = f"https://raw.githubusercontent.com/apache/airflow/constraints-{AIRFLOW_VERSION}/constraints-{PYTHON_VERSION}.txt"

!pip install "apache-airflow=={AIRFLOW_VERSION}" --constraint "{CONSTRAINT_URL}" pandas pyngrok > /dev/null 2>&1

# 3. Initialize Environment
os.environ["AIRFLOW_HOME"] = "/content/airflow"
# Fix specific SQLAlchemy error in Colab
os.environ["AIRFLOW__CORE__SQL_ALCHEMY_CONN"] = "sqlite:////content/airflow/airflow.db"
os.environ["AIRFLOW__CORE__LOAD_EXAMPLES"] = "False" # Don't load example DAGs (saves memory)

print("✅ Installation Successful.")

🧹 Cleaning up conflicting libraries...
⏳ Installing Apache Airflow (Robust Mode)...
✅ Installation Successful.


In [5]:
# @title 2. Initialize Database & Create Admin
import os

# 1. Initialize DB
print("⚙️ Initializing SQLite Database...")
!airflow db init > /dev/null 2>&1

# 2. Create Admin User
print("👤 Creating Admin User...")
!airflow users create \
    --username admin \
    --firstname System \
    --lastname Admin \
    --role Admin \
    --email admin@example.com \
    --password admin \
    > /dev/null 2>&1

# 3. Create DAGs folder
!mkdir -p /content/airflow/dags

print("✅ Airflow Configured.")

⚙️ Initializing SQLite Database...
👤 Creating Admin User...
✅ Airflow Configured.


In [6]:
# @title 3. Create the Orchestration DAG
from datetime import datetime, timedelta

dag_content = """
from airflow import DAG
from airflow.operators.python import PythonOperator
from airflow.operators.bash import BashOperator
from datetime import datetime, timedelta
import random
import time
import json

def generate_order(**context):
    order_id = f"ORD-{random.randint(10000, 99999)}"
    amount = random.randint(50, 500)
    print(f"🛒 Order Generated: {order_id} for ${amount}")
    context['ti'].xcom_push(key='order_data', value={'id': order_id, 'amount': amount})

def process_payment(**context):
    order_data = context['ti'].xcom_pull(key='order_data', task_ids='generate_order_task')
    print(f"💳 Processing payment for {order_data['id']}...")
    time.sleep(1)

    # 20% Chance of failure to show Orchestration Retry Logic
    if random.random() < 0.2:
        raise ValueError("❌ Payment Gateway Timeout!")

    print("✅ Payment Authorized.")

def check_inventory(**context):
    order_data = context['ti'].xcom_pull(key='order_data', task_ids='generate_order_task')
    print(f"📦 Checking warehouse for {order_data['id']}...")
    time.sleep(1)
    print("✅ Stock Allocated.")

def ship_order(**context):
    order_data = context['ti'].xcom_pull(key='order_data', task_ids='generate_order_task')
    tracking_num = f"TRK-{random.randint(100,999)}"
    print(f"🚚 Order {order_data['id']} Shipped! Tracking: {tracking_num}")

default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'start_date': datetime(2023, 1, 1),
    'retries': 2,
    'retry_delay': timedelta(seconds=2),
}

with DAG(
    'ecommerce_fulfillment_pipeline',
    default_args=default_args,
    description='A simulated E-Commerce Saga',
    schedule_interval=timedelta(seconds=30),
    catchup=False,
    tags=['production'],
) as dag:

    t1 = PythonOperator(task_id='generate_order_task', python_callable=generate_order)
    t2 = PythonOperator(task_id='process_payment_task', python_callable=process_payment)
    t3 = PythonOperator(task_id='check_inventory_task', python_callable=check_inventory)
    t4 = PythonOperator(task_id='ship_order_task', python_callable=ship_order)

    t1 >> t2 >> t3 >> t4
"""

with open("/content/airflow/dags/ecommerce_pipeline.py", "w") as f:
    f.write(dag_content)

print("✅ DAG file created.")

✅ DAG file created.


In [8]:
# @title 4. Start Server & Open Tunnel (With Health Check)
import subprocess
import time
import requests
from pyngrok import ngrok, conf

# --- PASTE YOUR TOKEN BELOW ---
NGROK_AUTH_TOKEN = "2lByyY668AQEl2B574HOizPVWze_     3roGxEj1UQdg2rXyx7tH8"
# ------------------------------

if NGROK_AUTH_TOKEN == "PASTE_YOUR_NGROK_TOKEN_HERE" or NGROK_AUTH_TOKEN == "":
    print("❌ You missed the Ngrok Token. Please paste it in the code above.")
else:
    # 1. Kill old processes
    !pkill -f airflow

    # 2. Start Scheduler
    print("💓 Starting Scheduler...")
    subprocess.Popen(["airflow", "scheduler"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # 3. Start Webserver
    print("🖥️ Starting Webserver (this takes ~30 seconds)...")
    subprocess.Popen(["airflow", "webserver", "--port", "8080"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # 4. HEALTH CHECK LOOP
    # We wait until localhost:8080 responds before creating the tunnel
    print("⏳ Waiting for Airflow to boot up...")
    server_ready = False
    for i in range(20): # Try for 100 seconds (20 * 5)
        try:
            r = requests.get("http://localhost:8080/health")
            if r.status_code == 200:
                server_ready = True
                break
        except:
            pass
        time.sleep(5)
        print(".", end="")

    if server_ready:
        conf.get_default().auth_token = NGROK_AUTH_TOKEN
        public_url = ngrok.connect(8080).public_url
        print("\n\n" + "="*50)
        print(f"🚀 AIRFLOW IS LIVE: {public_url}")
        print("="*50)
        print("Login: admin / admin")
        print("1. Toggle the switch ON for 'ecommerce_fulfillment_pipeline'")
        print("2. Click the DAG name -> Click 'Graph' tab")
        print("3. Click the 'Play' button (top right) to trigger manually")
    else:
        print("\n❌ Timed out waiting for Airflow to start. Check logs.")

💓 Starting Scheduler...
🖥️ Starting Webserver (this takes ~30 seconds)...
⏳ Waiting for Airflow to boot up...
....................
❌ Timed out waiting for Airflow to start. Check logs.
